# Boston Housing Dataset

## Setup


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)

COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age", "dis", "tax", "ptratio", "medv"]
#FEATURES = ["crim", "zn", "indus", "nox", "rm", "age", "dis", "tax", "ptratio"]
#LABEL = "medv"

FEATURES = COLUMNS[:-1]
LABEL = COLUMNS[-1]

print("Features: {}".format(FEATURES))
print("Label: {}".format(LABEL))

def get_input_fn(data_set, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.pandas_input_fn(
      x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
      y=pd.Series(data_set[LABEL].values),
      num_epochs=num_epochs,
      shuffle=shuffle)

url_prefix = "https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/examples/tutorials/input_fn/"

training_set = pd.read_csv(
    url_prefix+"boston_train.csv", 
    skipinitialspace=True,
    skiprows=1, 
    names=COLUMNS)

test_set = pd.read_csv(
    url_prefix+"boston_test.csv", 
    skipinitialspace=True,
    skiprows=1, 
    names=COLUMNS)

prediction_set = pd.read_csv(
    url_prefix+"boston_predict.csv", 
    skipinitialspace=True,
    skiprows=1, 
    names=COLUMNS)

feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

regressor = tf.estimator.DNNRegressor(
    feature_columns=feature_cols,
    hidden_units=[10, 10],
    model_dir="/tmp/boston_housing")

regressor.train(input_fn=get_input_fn(training_set), steps=5000)

evaluation = regressor.evaluate(
    input_fn=get_input_fn(test_set, num_epochs=1, shuffle=False))

print("Evaluation: {}".format(evaluation))

loss = evaluation["loss"]
print("Loss: {0:f}".format(loss))

y = regressor.predict(
    input_fn=get_input_fn(prediction_set, num_epochs=1, shuffle=False))

print("predict: {}".format(y))

for yy in y:
  print("Prediction {}".format(yy))
  
# .predict() returns an iterator of dicts; convert to a list and print
# predictions
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))


Features: ['crim', 'zn', 'indus', 'nox', 'rm', 'age', 'dis', 'tax', 'ptratio']
Label: medv
Evaluation: {'average_loss': 11.506675, 'global_step': 15000, 'loss': 1150.6675}
Loss: 1150.667480
predict: <generator object predict at 0x7f27780737d0>
Prediction {'predictions': array([33.473564], dtype=float32)}
Prediction {'predictions': array([17.696209], dtype=float32)}
Prediction {'predictions': array([23.382742], dtype=float32)}
Prediction {'predictions': array([35.1132], dtype=float32)}
Prediction {'predictions': array([14.393742], dtype=float32)}
Prediction {'predictions': array([18.381556], dtype=float32)}
Predictions: []
